### Imports 

In [1]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from sagemaker import get_execution_role
import sagemaker
import boto3
import os

In [2]:
sagemaker.__version__

'2.16.1'

### Essentials

In [3]:
role = get_execution_role()
session = boto3.Session()
sagemaker_session = sagemaker.Session()

s3 = session.resource('s3')
TF_FRAMEWORK_VERSION = '2.3.0'

BUCKET = 'cv-examples-892313895307' # USE YOUR ACCOUNT ID OR INITIALS AS SUFFIX
PREFIX = 'sign-language-clf'

### Load Local Data to S3

In [ ]:
!aws s3 cp ./DATA/SIGN_LANGUAGE/ s3://{BUCKET}/{PREFIX}/ --recursive

In [5]:
train_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/train', 
                            distribution='ShardedByS3Key')
val_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/validation', 
                           distribution='ShardedByS3Key')

In [6]:
val_input.__dict__

{'config': {'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
    'S3Uri': 's3://cv-examples-892313895307/sign-language-clf/validation',
    'S3DataDistributionType': 'ShardedByS3Key'}}}}

### Train Model on SageMaker

In [7]:
model_name = 'sign-lang-model-1'

estimator_parameters = {'entry_point':'sign_language_train.py',
                        'instance_type': 'ml.m5.2xlarge',
                        'instance_count': 2,
                        'model_dir': f'/opt/ml/model',
                        'role': role,
                        'output_path': f's3://{BUCKET}/{PREFIX}/out',
                        'base_job_name': f'cv-{model_name}',
                        'framework_version': TF_FRAMEWORK_VERSION,
                        'py_version': 'py37',
                        'script_mode': True}

estimator = TensorFlow(**estimator_parameters)

In [ ]:
%%time

estimator.fit({'train': train_input, 'val': val_input})

In [9]:
model_location = estimator.model_data
model_location

's3://cv-examples-892313895307/sign-language-clf/out/cv-sign-lang-model-1-2020-11-19-22-34-47-252/output/model.tar.gz'

In [10]:
output_2 = f's3://{BUCKET}/{PREFIX}/mme/model2.tar.gz'

In [11]:
!aws s3 cp {model_location} {output_2} 

copy: s3://cv-examples-892313895307/sign-language-clf/out/cv-sign-lang-model-1-2020-11-19-22-34-47-252/output/model.tar.gz to s3://cv-examples-892313895307/sign-language-clf/mme/model2.tar.gz


### Deploy Model

In [12]:
from sagemaker.tensorflow import TensorFlowModel

In [13]:
model = TensorFlowModel(model_data=output_2, 
                        role=role,  
                        name='sign-lang-model', 
                        sagemaker_session=sagemaker_session, 
                        framework_version=TF_FRAMEWORK_VERSION)

In [14]:
predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m5.xlarge', 
                         endpoint_name='cv-sign-lang-model-2-endpoint')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!

### Test Deployed Endpoint 

In [16]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [19]:
img_path = './DATA/SIGN_LANGUAGE/test/0/IMG_1290.JPG'
img = image.load_img(img_path, target_size=(224, 224))
X = image.img_to_array(img)
X = np.expand_dims(X, axis=0)

In [20]:
predictor.predict(X)

{'predictions': [[0.998930514,
   0.000134256523,
   3.58891761e-08,
   0.000903726614,
   2.22441243e-09,
   1.77045545e-09,
   4.53694304e-10,
   1.90589642e-08,
   5.41306422e-09,
   3.14411336e-05]]}